# 0.0 Setup e carregamento

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
import importlib
import feature_engineering 
importlib.reload(feature_engineering)
warnings.filterwarnings('ignore')
sys.path.append('../src')
from feature_engineering import calc_woe_iv

# Carregar dados limpos
df = pd.read_csv('../data/processed_data/df_cleaned.csv')
target_col = 'inadipl_90dias_ult2anos'

print("=" * 80)
print("🔧 FEATURE ENGINEERING - PARTE 1")
print("=" * 80)
print(f" Dataset inicial:")
print(f"   Shape: {df_fe.shape}")
print(f"   Features: {df_fe.shape[1] - 1} (excluindo target)")

# Criar cópia para não alterar original
df_fe = df.copy()

🔧 FEATURE ENGINEERING - PARTE 1
 Dataset inicial:
   Shape: (149234, 27)
   Features: 26 (excluindo target)


# 1.0 Derivation feature "Comprometimento_renda"

Essa variavel perdeu poder discriminatório após a limpeza de truncagem em 300%, os clientes ficaram num intervalo plausível, removendo distorçoes de outliers que inflavam os adimplentes e inadimplentes mas isso acabou "achatando" a variabilidade e fez o information valuate cair para 0.002 não conseguindo diferenciar risco de forma significativa.
Será usado o IV calculado com a métrica WOE para avaliar se a variável ganha força quando categorizada em faixas(bins) e revelar padrões escondidos. Isso ajuda a validar se vale a pena mantê-la no Projeto.

In [42]:
# função para calcular WOE e IV
#df_fe['target'] = 1 inadimp, 2 adimp.

# Definição dos bins 
bins = [0, 50, 100, 150, 200, 250, 300]

# Chamada correta
woe_result, iv_value = calc_woe_iv(
    df_fe,
    'comprometimento_renda',
    'inadipl_90dias_ult2anos',
    bins=bins
)

print(woe_result)
print("IV total:", iv_value)


                 count   sum  non_event       woe        iv
bin                                                        
(-0.001, 50.0]   10914   579      10335 -0.249271  0.004083
(50.0, 100.0]     2824   159       2665 -0.186331  0.000606
(100.0, 150.0]    2178   209       1969  0.389744  0.002627
(150.0, 200.0]    1302   114       1188  0.288856  0.000826
(200.0, 250.0]    1331    99       1232  0.111428  0.000116
(250.0, 300.0]  130685  8848     121837  0.010221  0.000092
IV total: 0.008349835192713826


O resultado mostra que o IV total é de 0.0083 é extremamente baixo.
Ou seja, não tem poder de separar entre inadimplentes e adimplentes.
Isso acontece porque mais de 130 mil desses dados estão na faixa/bins de 250 a 300 e não traz variabilidade para diferenciar risco.
Os Bins abaixo de 200 mostra diferenças mas é uma fração pequena da base, então o modelo acaba vendo quase todo mundo igual.
* Talvez separar esses dados acima de 250 como categoria especial para não perder info.
* Ou criar variaveis derivadas para introduzir novas proporções para que não fique concentradas nos Bins mais altos.

In [17]:
#----------------VARIAVEIS DERIVADAS-----------------------------
## 1. Utilização média por linha de crédito pessoal.
#Captura se o cliente concentra divídas em poucas linhas ou distribui melhor.
df_fe['utilizacao_media_linha'] = df_fe['utilizacao_credito'] / (df_fe['linhas_credito_abertas'] + 1e-6)

# 2. Comprometimento ajustado pela renda
df_fe['comprometimento_renda_ajustado'] = df_fe['divida_ratio'] / (df_fe['renda_mensal'] + 1e-6)


# 3. Índice de severidade de atrasos
df_fe['indice_severidade_atrasos'] = ( df_fe['atrasos_30dias']*1 + df_fe['atrasos_90dias']*3 )


# 4. Intensidade de crédito por idade
df_fe['intensidade_credito_idade'] = df_fe['linhas_credito_abertas'] / (df_fe['idade'] + 1e-6)


# 5. Proporção de crédito imobiliário
df_fe['proporcao_credito_imobiliario'] = df_fe['emprestimos_imobiliarioss'] / (df_fe['linhas_credito_abertas'] + 1e-6)


# 6. Renda per capita
df_fe['renda_per_capita'] = df_fe['renda_mensal'] / (df_fe['dependentes'] + 1)



In [18]:
# Lista de variáveis derivadas
variaveis_derivadas = [
    'utilizacao_media_linha',
    'comprometimento_renda_ajustado',
    'indice_severidade_atrasos',
    'intensidade_credito_idade',
    'proporcao_credito_imobiliario',
    'renda_per_capita'
]

# Definição de bins (ajuste conforme cada variável)
bins_dict = {
    'utilizacao_media_linha': [0, 0.25, 0.5, 1, 2, 5, 10],
    'comprometimento_renda_ajustado': [0, 1e-05, 5e-05, 1e-04, 5e-04, 1e-03],
    'indice_severidade_atrasos': [0, 1, 2, 3, 5, 10],
    'intensidade_credito_idade': [0, 0.1, 0.2, 0.5, 1, 2],
    'proporcao_credito_imobiliario': [0, 0.25, 0.5, 0.75, 1],
    'renda_per_capita': [0, 500, 1000, 2000, 5000, 10000, 20000]
}

# Loop para calcular IV de todas
iv_results = {}
for var in variaveis_derivadas:
    iv_results[var] = calc_woe_iv(df_fe, var, 'inadipl_90dias_ult2anos', bins_dict[var])

# Converter em DataFrame para tabela comparativa
iv_table = pd.DataFrame.from_dict(iv_results, orient='index', columns=['IV'])
iv_table = iv_table.sort_values(by='IV', ascending=False)

print(iv_table)


                                      IV
indice_severidade_atrasos       0.981132
renda_per_capita                0.585419
utilizacao_media_linha          0.205095
comprometimento_renda_ajustado  0.088908
intensidade_credito_idade       0.022117
proporcao_credito_imobiliario   0.011416


In [5]:
df_fe[[
    'utilizacao_media_linha',
    'comprometimento_renda_ajustado',
    'indice_severidade_atrasos',
    'intensidade_credito_idade',
    'proporcao_credito_imobiliario',
    'renda_per_capita'
]]

,utilizacao_media_linha,comprometimento_renda_ajustado,indice_severidade_atrasos,intensidade_credito_idade,proporcao_credito_imobiliario,renda_per_capita
0,0.058931,8.804825e-05,2,0.288889,0.461538,3040.000000
1,0.239300,4.688462e-05,0,0.100000,0.000000,1300.000000
2,0.329100,2.797502e-05,3,0.052632,0.000000,3042.000000
3,0.046760,1.090909e-05,0,0.166667,0.000000,3300.000000
4,0.129600,4.980000e-07,1,0.142857,0.142857,50000.000000
...,...,...,...,...,...,...
149229,0.010175,1.071905e-04,0,0.054054,0.250000,2100.000000
149230,0.074925,1.283309e-04,0,0.090909,0.250000,1861.333333
149231,0.013667,6.151645e-01,0,0.310345,0.055556,6291.000000
149232,0.000000,0.000000e+00,0,0.133333,0.000000,5716.000000


In [21]:
# Lista de variáveis originais
variaveis_originais = [
    'utilizacao_credito',
    'idade',
    'atrasos_30dias',
    'divida_ratio',
    'renda_mensal',
    'linhas_credito_abertas',
    'atrasos_90dias',
    'emprestimos_imobiliarioss',
    'dependentes'
]

# Definição de bins para originais (ajuste conforme distribuição real)
bins_originais = {
    'utilizacao_credito': [0, 0.25, 0.5, 1, 2, 5, 10],
    'idade': [18, 25, 35, 45, 60, 80, 100],
    'atrasos_30dias': [0, 1, 2, 3, 5, 10],
    'divida_ratio': [0, 0.25, 0.5, 1, 2, 5],
    'renda_mensal': [0, 1000, 2000, 5000, 10000, 20000],
    'linhas_credito_abertas': [0, 2, 5, 10, 20, 50],
    'atrasos_90dias': [0, 1, 2, 3, 5, 10],
    'emprestimos_imobiliarioss': [0, 1, 2, 3, 5, 10,20,30],
    'dependentes': [0, 1, 2, 3, 5, 10]
}

# Extrair apenas o IV das originais (segunda posição da tupla)
iv_results_originais_iv = {var: iv_results_originais[var][1] for var in iv_results_originais}

# Para derivadas, se já são números, não precisa acessar índice
iv_results_iv = {var: iv_results[var] for var in iv_results}

# Juntar originais e derivadas
iv_results_total = {**iv_results_originais_iv, **iv_results_iv}

# Converter em DataFrame
iv_table_total = pd.DataFrame.from_dict(iv_results_total, orient='index', columns=['IV'])
iv_table_total = iv_table_total.sort_values(by='IV', ascending=False)

print(iv_table_total)



                                      IV
utilizacao_credito              1.080395
indice_severidade_atrasos       0.981132
renda_per_capita                0.585419
atrasos_30dias                  0.455241
atrasos_90dias                  0.454066
idade                           0.229175
dependentes                     0.213085
utilizacao_media_linha          0.205095
comprometimento_renda_ajustado  0.088908
renda_mensal                    0.085136
linhas_credito_abertas          0.084017
divida_ratio                    0.068744
intensidade_credito_idade       0.022117
emprestimos_imobiliarioss       0.022051
proporcao_credito_imobiliario   0.011416


In [25]:
# Definir limiar de IV (ajuste conforme necessidade)
iv_threshold = 0.05

# Selecionar variáveis com IV acima do limiar
selected_vars = iv_table_total[iv_table_total['IV'] > iv_threshold].index.tolist()

print("Variáveis selecionadas:", selected_vars)

# Criar dataset filtrado apenas com as variáveis selecionadas
df_features_selected = df_fe[selected_vars]

print("Shape do dataset filtrado:", df_features_selected.shape)

Variáveis selecionadas: ['utilizacao_credito', 'indice_severidade_atrasos', 'renda_per_capita', 'atrasos_30dias', 'atrasos_90dias', 'idade', 'dependentes', 'utilizacao_media_linha', 'comprometimento_renda_ajustado', 'renda_mensal', 'linhas_credito_abertas', 'divida_ratio']
Shape do dataset filtrado: (149234, 12)


In [26]:
df_features_selected


,utilizacao_credito,indice_severidade_atrasos,renda_per_capita,atrasos_30dias,atrasos_90dias,idade,dependentes,utilizacao_media_linha,comprometimento_renda_ajustado,renda_mensal,linhas_credito_abertas,divida_ratio
0,0.7661,2,3040.000000,2,0,45,2,0.058931,8.804825e-05,9120.0,13,0.8030
1,0.9572,0,1300.000000,0,0,40,1,0.239300,4.688462e-05,2600.0,4,0.1219
2,0.6582,4,3042.000000,1,1,38,0,0.329100,2.797502e-05,3042.0,2,0.0851
3,0.2338,0,3300.000000,0,0,30,0,0.046760,1.090909e-05,3300.0,5,0.0360
4,0.9072,1,50000.000000,1,0,49,0,0.129600,4.980000e-07,50000.0,7,0.0249
...,...,...,...,...,...,...,...,...,...,...,...,...
149229,0.0407,0,2100.000000,0,0,74,0,0.010175,1.071905e-04,2100.0,4,0.2251
149230,0.2997,0,1861.333333,0,0,44,2,0.074925,1.283309e-04,5584.0,4,0.7166
149231,0.2460,0,6291.000000,0,0,58,0,0.013667,6.151645e-01,6291.0,18,3870.0000
149232,0.0000,0,5716.000000,0,0,30,0,0.000000,0.000000e+00,5716.0,4,0.0000
